# CSIE 5452, Fall 2019 — Homework 1

In [1]:
import sys
import numpy as np
from pandas.core.frame import DataFrame

sys.setrecursionlimit(9000000)

## Functions 

### 1. Read file and preprocess data 

In [2]:
def preprocess_data(file_dir):
    
    with open(file_dir, 'r') as f:
        dat = [line.rstrip('\n') for line in f]
    msg_num = int(dat[0])
    one_bit_trans = float(dat[1])
              
    temp = []
    for i in range (len(dat) - 2):
        temp.append(dat[i+2])
    
    data = []
    i = 0
    for d in temp:
        data.append(d.split(' '))
        if '' in data[i]:
            data[i].remove('')
        for j, item  in  enumerate(data[i]):
            try:
                data[i][j] = int(data[i][j])
            except ValueError:
                data[i][j] = float(data[i][j])
        i+=1
    
    return msg_num, one_bit_trans, data

### 2. Compute right hand side 

In [3]:
def RHS(i, Qi, Bi, tao, data):
    
    temp = 0
    for n in range (i):
        temp += ((Qi + tao)/data[n][2])*data[n][1]
    rhs = Bi + temp
    
    return rhs

### 3. Compute Qi

In [4]:
def calQi(i, q, Bi, Ci, Ti, tao, data):
    
    rhs = RHS(i, q, Bi, tao, data)
    if rhs + Ci >Ti:
        return 'nan'
    elif rhs == q:
        return q
    else:
        return calQi(i, rhs, Bi, Ci, Ti, tao, data)

### 4. Compute blocking time

In [5]:
def calBi(i, data):
    
    blocking_time = data[i][1]
    for j in range(len(data) - (i+1)):
        if data[i+j][1] > blocking_time:
            blocking_time = data[i+j][1]
    
    return blocking_time

### 5. Compute worst-case response time

In [6]:
def calRi(i, Bi, tao, data):
    
    if i == 0:
        Q = Bi
        R = Q + data[i][1]
        return R
    else:
        Q = calQi(i, Bi, Bi, data[i][1], data[i][2], tao, data)
        if Q == 'nan':
            return print('The system is not schedulable')
        else:
            R = Q + data[i][1]        
            return R

## Main 

In [7]:
FILE_DIR = 'Input.dat'

In [8]:
MSG_NUM, TAO, DATA = preprocess_data(FILE_DIR)
dataframe = DataFrame(DATA)
dataframe.rename(columns={0:'Priority',1:'Trans. Time',2:'Period'},inplace=True)

print ('Message quantity: '+ str(MSG_NUM) + ', Transmission of one bit: '+ str(TAO))
dataframe

Message quantity: 17, Transmission of one bit: 0.002


,Priority,Trans. Time,Period
0,0,0.52,50
1,1,0.60,5
2,2,0.52,5
3,3,0.60,5
4,4,0.52,5
5,5,0.60,5
6,6,0.92,10
7,7,0.52,10
8,8,0.60,10
9,9,0.68,10


In [9]:
Ri_list = []
for i, item in enumerate(DATA):
    Bi = calBi(i, DATA)
    Ri = calRi(i, Bi, TAO, DATA)
    Ri_list.append(Ri)
    print ('Message μ'+ str(i) + ' worst-case response time: ' + str(Ri))

Ri_list

Message μ0 worst-case response time: 1.44
Message μ1 worst-case response time: 1.5296895715440582
Message μ2 worst-case response time: 1.5782575896964122
Message μ3 worst-case response time: 1.8022842215256007
Message μ4 worst-case response time: 1.9461288723667907
Message μ5 worst-case response time: 2.3003633677991138
Message μ6 worst-case response time: 3.1049070208728646
Message μ7 worst-case response time: 2.8298932038834943
Message μ8 worst-case response time: 3.342956772334292
Message μ9 worst-case response time: 4.179838235294114
Message μ10 worst-case response time: 5.611582887700525
Message μ11 worst-case response time: 6.232137931034473
Message μ12 worst-case response time: 5.700370820668679
Message μ13 worst-case response time: 5.913569620253163
Message μ14 worst-case response time: 6.305062706270615
Message μ15 worst-case response time: 4.849231330899434
Message μ16 worst-case response time: 4.8679999999999914


[1.44,
 1.5296895715440582,
 1.5782575896964122,
 1.8022842215256007,
 1.9461288723667907,
 2.3003633677991138,
 3.1049070208728646,
 2.8298932038834943,
 3.342956772334292,
 4.179838235294114,
 5.611582887700525,
 6.232137931034473,
 5.700370820668679,
 5.913569620253163,
 6.305062706270615,
 4.849231330899434,
 4.8679999999999914]

## Homework Question 1

In [10]:
Q1_DATALIST = [[0, 10, 50], [1, 30, 200], [2, 20, 100]]
TAO = 0.1

for i, item in enumerate(Q1_DATALIST):
    Bi = calBi(i, Q1_DATALIST)
    Ri = calRi(i, Bi, TAO, Q1_DATALIST)
    Ri_list.append(Ri)
    print ('Message μ'+ str(i) + ' worst-case response time: ' + str(Ri))

Message μ0 worst-case response time: 40
Message μ1 worst-case response time: 67.525
Message μ2 worst-case response time: 50.823076923076925
